<a href="https://colab.research.google.com/github/Kambey-san/NLP_base/blob/HW_lesson_16/KAA_NLP_HW_16_TCB_V8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Курсовой проект "telegram chat-bot"
### Данные:
<a href='https://drive.google.com/file/d/1t3ha_c-mkmWhfvtzc77GNUFOIXCH8ZOq/view?usp=drive_link'>Otvety.txt</a>,

<a href='https://drive.google.com/file/d/1eLD7icgzaZhaUP9K58A41VblxgSFH2rp/view?usp=drive_link'>ProductsDataset</a>,

<a href='https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.1.0/ru_core_news_sm-3.1.0.tar.gz'>ru_core_news_sm</a>.

In [ ]:
# # Mounting drive

# import os, sys
# from google.colab import drive
# drive.mount('/content/drive')
# colab_path = '/content/notebooks'
# os.symlink('/content/drive/My Drive/Colab Notebooks', colab_path)
# sys.path.insert(0,colab_path)

Mounted at /content/drive


In [3]:
!pip install -q dialogflow gensim annoy tqdm stop_words pymorphy2

In [4]:
!pip install -q python-telegram-bot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.9/604.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [5]:
!pip install -q pandarallel

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
import annoy
import mmap
import string
import pickle
import os
import re
from telegram.ext import Updater, CommandHandler, MessageHandler, filters, CallbackContext
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText
from tqdm.notebook import tqdm
from pandarallel import pandarallel

tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
link = 'https://drive.google.com/file/d/1Y8mc9azsGEcNfZFo_nHPE5jxRpWN-UW3/view?usp=drive_link'

# to get the id part of the file
id_test = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id_test})
downloaded.GetContentFile('Otvety.txt')

In [9]:
def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

### Преобразование файла вопросов-ответов в строчный вид

In [10]:
if not os.path.isfile('prepared_answers.txt'):

    question = None
    written = False

    with open("prepared_answers.txt", "w", encoding='utf-8') as fout:
        with open("Otvety.txt", "r") as fin:
            for line in tqdm(fin):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()
                    continue

0it [00:00, ?it/s]

### Препроцессинг текста

In [11]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [12]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

### Обработка текста

In [13]:
sentences = []
c = 0

file_path_from = 'prepared_answers.txt'
file_path_to = 'Otvety2.txt'

if not os.path.isfile(file_path_to):

    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'w') as fileto:
        with open(file_path_from) as filefrom:
            for k in tqdm(range(N)):
                line = filefrom.readline()
                if line == '': break
                spls = preprocess_txt(line)
                sentences.append(spls)
                c += 1
                if c > 500000000:
                    break
                fileto.write(' '.join(spls)+'\n')
    filefrom.close()
    fileto.close()

  0%|          | 0/307926 [00:00<?, ?it/s]

### Загрузка результата

In [14]:
sentences = []

file_path_from = 'Otvety2.txt'
if os.path.isfile(file_path_from):
    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'r') as filefrom:
        for k in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            sentences.append(line.split())
    filefrom.close()

  0%|          | 0/307926 [00:00<?, ?it/s]

In [15]:
vec = []
_ = [vec.extend(x)  for x in sentences[:100]]
vec = list(set(vec))
vec.sort()
vec[20:30]

['1542',
 '1548',
 '1588',
 '16ть',
 '1890—1907liul',
 '1898',
 '18мкака',
 '1914',
 '1мми',
 '2']

### Обучение модели FastText

In [16]:
file_path_from = 'ft_model'
if not os.path.isfile(file_path_from):

    sentences = [i for i in tqdm(sentences) if len(i) > 2]
    modelFT = FastText(sentences=sentences, vector_size=100, window=3, min_count=1, epochs=10)
    modelFT.save("ft_model")

  0%|          | 0/307926 [00:00<?, ?it/s]

### Загрузка модели

In [17]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

### Индексы "вопрос-ответ"

In [18]:
file_path_from = 'speaker.ann'
if not os.path.isfile(file_path_from):

    modelFT = FastText.load("ft_model")
    ft_index = annoy.AnnoyIndex(100 ,'angular')

    index_map = {}
    counter = 0
    with open("Otvety2.txt", "r") as f:
        for line in tqdm(f):
            n_ft = 0
            spls = line.split("\t")
            index_map[counter] = spls[0]
            question = preprocess_txt(spls[0])
            vector_ft = np.zeros(100)
            for word in question:
                if word in modelFT.wv:
                    vector_ft += modelFT.wv[word]
                    n_ft += 1
            if n_ft > 0:
                vector_ft = vector_ft / n_ft
            ft_index.add_item(counter, vector_ft)

            counter += 1

            if counter > 50_000:
                break

    ft_index.build(10)
    ft_index.save('speaker.ann')

    with open("index_speaker.pkl", "wb") as f:
        pickle.dump(index_map, f)

0it [00:00, ?it/s]

In [19]:
#  Загрузка индексов
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')
index_map = pd.read_pickle("index_speaker.pkl")

In [20]:
np.random.permutation(100)

array([42, 21, 61, 35, 63,  0, 57, 60, 83, 34, 98,  7, 82, 23, 85, 26, 15,
       24, 74, 97,  9, 77, 18, 78, 71, 31, 25, 58, 96, 51, 50, 40, 33, 67,
       36, 99, 80, 95, 62, 38, 12, 43, 84, 92, 69, 65, 27,  6, 70,  5, 52,
       72, 89, 54, 39, 22, 93,  3, 44, 11, 29, 32, 14, 13,  2, 19, 30, 91,
       49, 46, 66, 10, 53, 47, 17,  1, 55, 86, 48, 81, 68, 75, 20, 79, 87,
       28, 45, 88, 41, 59, 73, 56, 94,  8, 16, 37, 64,  4, 90, 76])

In [21]:
#  Пример получения индексов
a = ft_index.get_nns_by_vector(np.random.permutation(100), 5, include_distances=True)
a

([48342, 35322, 14924, 29810, 32173],
 [1.2487246990203857,
  1.2556674480438232,
  1.2602990865707397,
  1.2635695934295654,
  1.2640094757080078])

In [22]:
[index_map[x] for x in a[0]]

['делать сделать жена отдать дядя идти\n',
 'достать бывший жена пипец досталачто сделать вестись провокация\n',
 'минимум уменьшить алименты можноль заставить бывший жена отчитываться алименты приколоть мдатут приколоть\n',
 'постоянно плач уезжать родственник перебороть плакать мужик\n',
 'жена изменить выгнать\n']

### Модель продуктовых данных

In [23]:
shop_data = pd.read_csv("ProductsDataset.csv")
#shop_data = shop_data.iloc[:5000, :]

shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].progress_apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [25]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
# Вопрос-ответный домен
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in tqdm(idxs)]
# Продуктовый домен
positive_texts = [" ".join(val) for val in tqdm(shop_data['text'].values)]

  0%|          | 0/25515 [00:00<?, ?it/s]

  0%|          | 0/35548 [00:00<?, ?it/s]

In [26]:
# ВО = 0, Прод = 1

dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2,
                                                    stratify=labels, random_state=13)

In [28]:
# Модель

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [29]:
# Качество

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9833783673135184

In [30]:
# + IDF взвешивание (для каждого слова поиск IDF веса)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [31]:
np.mean(tfidf_vect.idf_)

10.770331712499575

In [32]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [33]:
len(idfs.keys())

173996

In [34]:
list(idfs.keys())[:5]

['ребята', 'помогитепожалуйст', 'выписать', 'вначале', 'предложение']

In [35]:
list(idfs.values())[:5]

[9.157472798475213,
 6.847770237712305,
 10.410235766970581,
 11.103382947530527,
 11.103382947530527]

In [36]:
# Индексы продуктовых данных

file_path_from = 'shop.ann'
if not os.path.isfile(file_path_from):


    ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_shop = {}
    counter = 0

    for i in tqdm(range(len(shop_data))):
        n_ft = 0
        index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
        vector_ft = np.zeros(100)
        for word in shop_data.loc[i, "text"]:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_shop.add_item(counter, vector_ft)
        counter += 1

    ft_index_shop.build(10)
    ft_index_shop.save('shop.ann')

    file_path_from = 'index_shop.pkl'
    if not os.path.isfile(file_path_from):

        with open("index_shop.pkl", "wb") as f:
            pickle.dump(index_map_shop, f)

  0%|          | 0/35548 [00:00<?, ?it/s]

In [37]:
# Загрузка индексов

midf = np.mean(tfidf_vect.idf_)

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann')

index_map_shop = pd.read_pickle("index_shop.pkl")

In [38]:
# Основная функция преобразования текста в вектор х100

def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [39]:
# Пример получения индекса

ft_index_shop.get_nns_by_vector(np.ones(100)*20, 5, include_distances=True)

([15698, 21895, 2965, 32313, 35057],
 [1.2124146223068237,
  1.2267783880233765,
  1.2727690935134888,
  1.273189663887024,
  1.2793858051300049])

### Прогноз погоды

In [41]:
# Проверка языка текста

import re

def is_english(text):
    return bool(re.search(r'[a-zA-Z]', text))

In [42]:
# Список ключевых слов для определения запроса о погоде

list_for_weather = ['погода','солнечно', 'осадка', 'дождь', 'снег', 'ливень', 'зонт']
list_for_translation = ['перевод', 'английский']

In [43]:
# Проверка есть ли совпадение со списком ключевых слов

def is_in_list(text, list):
    for word in text:
        if word in list:
            return True
            break
    return False

In [44]:
# Запрос прогноза погоды

import requests
import json

key = '9ead4b09f8ab46fb9ce105955220407' # Токен https://www.weatherapi.com/

def get_weather(city):
    url = f'http://api.weatherapi.com/v1/current.json?key={key}&q={city}&lang=ru'
    requests.get(url)
    response = requests.get(url)
    json_data = json.loads(response.text)

    city = json_data['location']['name']
    time = json_data['location']['localtime'].split(' ')[1]
    temp = json_data['current']['temp_c']
    temp_fl = json_data['current']['feelslike_c']
    condition = json_data['current']['condition']['text']
    wind = json_data['current']['wind_mph']
    uv = json_data['current']['uv']

    msg = f'В городе {city} сейчас {time}. О погоде: {condition}, температура {temp} градусов, ощущается как {temp_fl} градусов, скорость ветра {wind} м/ч, уровень ультрафиолета {uv}'

    return msg

In [46]:
import spacy

In [47]:
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.1.0/ru_core_news_sm-3.1.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 12.4 MB/s eta 0:00:00
  Created wheel for ru-core-news-sm: filename=ru_core_news_sm-3.1.0-py3-none-any.whl size=16086391 sha256=ef29fadbb4a7f7ed32bae604592432c01fa9f6729c2dacd786595781ea07aa71
  Stored in directory: /root/.cache/pip/wheels/5c/20/2c/4830d46ba72a770a01170c2a94938f4fa770ad7781340ccc4b
Successfully built ru-core-news-sm
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstall

In [48]:
# Определение города

nlp = spacy.load("ru_core_news_sm")

def location_recognition(text):
    locs = []
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'LOC':
            locs.append(ent.text)
    locs = [morpher.parse(i.lower())[0].normal_form for i in locs]
    return locs

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'ru_core_news_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [49]:
# Погода в заданном городе

def forecast(text):
    city = location_recognition(text)
    try:
        forecast = get_weather(city)
    except:
        forecast = 'Не понимаю тебя'

    return forecast

### Бот

In [ ]:
# заменить на свой токен
updater = Updater("6376866337:AAEM0fQRN4Jp0IzcY4d-zNaUp-kGMkZUecM", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, пообщаемся?')

def textMessage(update, context):

    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)

    # ПРОД
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return

    # QA
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)

    #
    if distances[0] > 100.5:
        print(distances[0])
        context.bot.send_message(chat_id=update.message.chat_id, text="Не понимаю ((")
        return

    # Вопрос-Ответ
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])

start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()